## Big Data ML AI 7 - José Manuel Guzmán Gutiérrez
# Práctica Final NLP
### Ejercicio 2 -  Etapa de preprocesado de texto

-------------

### Cargamos los datos (Repetimos los primeros pasos del anterior ejercicio)

In [1]:
import pandas as pd
import glob
import os

In [2]:
data_reviews = pd.read_json('./data/reviews_Grocery_and_Gourmet_Food_5.json.gz', lines=True, compression='gzip')

In [3]:
data_reviews[:100]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1VEELTKS8NLZB,616719923X,Amazon Customer,"[0, 0]",Just another flavor of Kit Kat but the taste i...,4,Good Taste,1370044800,"06 1, 2013"
1,A14R9XMZVJ6INB,616719923X,amf0001,"[0, 1]",I bought this on impulse and it comes from Jap...,3,"3.5 stars, sadly not as wonderful as I had hoped",1400457600,"05 19, 2014"
2,A27IQHDZFQFNGG,616719923X,Caitlin,"[3, 4]",Really good. Great gift for any fan of green t...,4,Yum!,1381190400,"10 8, 2013"
3,A31QY5TASILE89,616719923X,DebraDownSth,"[0, 0]","I had never had it before, was curious to see ...",5,Unexpected flavor meld,1369008000,"05 20, 2013"
4,A2LWK003FFMCI5,616719923X,Diana X.,"[1, 2]",I've been looking forward to trying these afte...,4,"Not a very strong tea flavor, but still yummy ...",1369526400,"05 26, 2013"
...,...,...,...,...,...,...,...,...,...
95,A1R19K6O4IAF5W,B0000CDEPD,Terry,"[2, 2]",Very good vanilla extract and organic too! Thi...,5,Much easier than making your own.,1365292800,"04 7, 2013"
96,AY8UH4COUYMGZ,B0000CDEPD,ttt,"[1, 2]","This bottle is a steal, at Whole Foods (at lea...",5,The Only Vanilla I'll Buy From Now On!,1373673600,"07 13, 2013"
97,A1RVOM92NT7WTM,B0000CDEPD,Vickie D.,"[0, 0]",Very good and even better knowing it is organi...,5,Added it to my subscribe,1398643200,"04 28, 2014"
98,A31EZQGRVKKYAZ,B0000CETGM,J. Hesse,"[1, 2]",Jamaica Blue Mountain Coffee is my absolute fa...,5,My favorite coffee,1369008000,"05 20, 2013"


In [4]:
data_reviews.shape

(151254, 9)

In [5]:
data_reviews = pd.DataFrame(data_reviews)

In [6]:
data_reviews.dtypes

reviewerID        object
asin              object
reviewerName      object
helpful           object
reviewText        object
overall            int64
summary           object
unixReviewTime     int64
reviewTime        object
dtype: object

Contamos con 151254 registros para 9 columnas de información

Nos quedamos con las columnas _**summary**_ , _**reviewText**_ , _**reviewerName**_ y _**overall**_ las que consideramos importantes para nuestro análisis de sentimiento.

In [7]:
data_reviews = data_reviews.drop(['reviewerID'], axis=1)
data_reviews = data_reviews.drop(['asin'], axis=1)
data_reviews = data_reviews.drop(['helpful'], axis=1)
data_reviews = data_reviews.drop(['unixReviewTime'], axis=1)
data_reviews = data_reviews.drop(['reviewTime'], axis=1)

In [8]:
data_reviews.head()

,reviewerName,reviewText,overall,summary
0,Amazon Customer,Just another flavor of Kit Kat but the taste i...,4,Good Taste
1,amf0001,I bought this on impulse and it comes from Jap...,3,"3.5 stars, sadly not as wonderful as I had hoped"
2,Caitlin,Really good. Great gift for any fan of green t...,4,Yum!
3,DebraDownSth,"I had never had it before, was curious to see ...",5,Unexpected flavor meld
4,Diana X.,I've been looking forward to trying these afte...,4,"Not a very strong tea flavor, but still yummy ..."


In [9]:
first_column = data_reviews.pop('summary')
data_reviews.insert(1, 'summary', first_column)
data_reviews.head()

,reviewerName,summary,reviewText,overall
0,Amazon Customer,Good Taste,Just another flavor of Kit Kat but the taste i...,4
1,amf0001,"3.5 stars, sadly not as wonderful as I had hoped",I bought this on impulse and it comes from Jap...,3
2,Caitlin,Yum!,Really good. Great gift for any fan of green t...,4
3,DebraDownSth,Unexpected flavor meld,"I had never had it before, was curious to see ...",5
4,Diana X.,"Not a very strong tea flavor, but still yummy ...",I've been looking forward to trying these afte...,4


### Pipeline de preprocesado

En base al pipeline del notebook 3, customizamos nuestro propio proceso en una única función para que nos devuelva un string de palabras separadas por espacio.

In [20]:
import unicodedata
from num2words import num2words

from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

tokenizer = RegexpTokenizer(r'\w+')
sw_list = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 

def nltk_cleaner(text, tokenizer, sw_list, lemmatizer):
    clean_text = list()
    text_ = list()

    for sentence in text:
        text_.append(unicodedata.normalize('NFKD', sentence).encode('ascii', 'ignore').decode('utf-8', 'ignore'))

        for word in tokenizer.tokenize(sentence):
            #No elimina StopWords y no sé por qué
            if word not in sw_list:
                word_no_SW = ''.join(word)
                clean_word = lemmatizer.lemmatize(word_no_SW).lower().strip()

                if clean_word.isdigit():
                    clean_word = num2words(clean_word, lang='en')

                clean_text.append(clean_word)
            clean_sentences = ' '.join(clean_text)
        text_total = ''.join(clean_sentences)
        
    #return text_total    // con este return no aparecen filtradas las StopWords, 
    # Se implementa en el return una nueva función    
    return ' '.join([word for word in text_total.split() if word not in sw_list])

tokenizer = RegexpTokenizer(r'\w+')
sw_list = stopwords.words('english')
lemmatizer = WordNetLemmatizer()   

data_rev = nltk_cleaner(data_reviews['reviewText'][:1000], tokenizer, sw_list, lemmatizer)
print(len(data_rev))
print(data_rev[:1000])

313882
another flavor kit kat taste unique bit different thing bothersome price thought bit expensive bought impulse come japan amused family weird stamp marking package fun said would take month arrive take long hoping interesting taste family tasted bit le flavorful weaker standard milk chocolate kit kat green tea flavor subtle sugar tasted sweet wafer crispy good tasted bit anemic u happy bought need buy really good great gift fan green tea expensive purchase candy across sea never curious see like smooth great subtle good flavor ordering plan make routine looking forward trying hearing popular japan among kit kat fan well recommend ordering warm weather melt become smushy ordered mine right summer began bit mushy let solidify room temp afterwards tried tasted fine expecting stronger green tea matcha flavor actually quite subtle outer coating creamy overly sugary liked overall say insanely good definitely yummy treat kit kat good looking strong green tea flavor disappointed green te

In [ ]:
Sentences = ' '.join([word for word in text_total.split() if word not in sw_list])